In [ ]:
# import os
# import pprint
# import yfinance as yf
# from pydantic import BaseModel, Field
# from typing import Literal
# from dotenv import load_dotenv

# # --- 1. Import the LangChain 1.0 Components ---
# from langchain.agents import create_agent
# # --- 
# # --- ⬇️ FIX #1: Import ToolStrategy instead ⬇️ ---
# # ---
# from langchain.agents.structured_output import ToolStrategy 
# from langchain.tools import tool
# from langchain.chat_models import init_chat_model

# # --- 2. Load API Key ---
# load_dotenv()

# if not os.getenv("GOOGLE_API_KEY"):
#     raise EnvironmentError("GOOGLE_API_KEY not found in .env file. Please add it.")

# # --- 3. Define Your REAL Tools (Perfect, no changes) ---

# @tool
# def get_stock_price(ticker: str) -> str:
#     """Get the latest stock price of a company (live) given its ticker."""
#     print(f"\n[Tool Called: get_stock_price(ticker='{ticker}')]")
#     try:
#         stock = yf.Ticker(ticker)
#         price = stock.info.get('currentPrice')
#         if not price:
#             data = stock.history(period="1d")
#             if data.empty:
#                 return f"⚠️ No recent price data found for {ticker}."
#             price = data["Close"].iloc[-1]
        
#         return f"Current price of {ticker} is ${price:.2f}"
#     except Exception as e:
#         return f"❌ Error fetching stock price for {ticker}: {e}"

# @tool
# def get_company_revenue(ticker: str) -> str:
#     """Fetch the latest annual revenue in USD for a company given its ticker."""
#     print(f"\n[Tool Called: get_company_revenue(ticker='{ticker}')]")
#     try:
#         stock = yf.Ticker(ticker)
#         fin = stock.financials
#         if fin.empty or "Total Revenue" not in fin.index:
#             return f"⚠️ Revenue data not available for {ticker}."
#         latest = fin.loc["Total Revenue"].iloc[0]
#         return f"Latest annual revenue of {ticker} is ${latest / 1_000_000_000:.2f} B"
#     except Exception as e:
#         return f"❌ Error fetching revenue for {ticker}: {e}"

# tools = [get_stock_price, get_company_revenue]

# # --- 4. Define Your "Consultant Report" Output Object (Perfect, no changes) ---

# class FinancialDetails(BaseModel):
#     ticker: str = Field(description="The stock ticker symbol.")
#     price_usd: float = Field(description="The current stock price in USD.")
#     company: str = Field(description="The name of the company.")
#     latest_year_revenue_usd: float = Field(description="The latest year's revenue in USD, as a number (e.g., 394300000000).")

# class StructuredRecommendation(BaseModel):
#     strategic_move: str = Field(description="The high-level strategic action to consider.")
#     rationale: str = Field(description="The brief justification for this move, linking it to the strategic insight.")
#     first_actionable_step: str = Field(description="A single, concrete next step to begin this strategy.")

# class ConsultantReport(BaseModel):
#     data_summary: str = Field(description="A brief, professional summary of the raw data found.")
#     structured_data: FinancialDetails = Field(description="The hard financial data points.")
#     strategic_insight: str = Field(description="Your core analysis. What does this data *mean* in a business context?")
#     recommendation: StructuredRecommendation = Field(description="The structured, actionable recommendation.")
#     confidence: Literal["low", "medium", "high"] = Field(description="Your confidence level in the analysis.")

# # --- 5. Initialize the Gemini Model (Perfect, no changes) ---
# llm = init_chat_model(
#     "google_genai:gemini-2.0-flash", 
#     temperature=0
# )

# # --- 6. Create the LangChain 1.0 Agent ---
# system_prompt = (
#     "You are a top-tier business analytics and strategy consultant. "
#     "Your client has asked for a quick analysis on a company ticker. "
#     "Your job is to use the provided tools to gather raw data for that ticker. "
#     "Then, synthesize that data into a high-level strategic report. "
#     "DO NOT just report the numbers. Your real value is in providing sharp insights. "
#     "You must fill out all fields in the required JSON format, including the "
#     "full 'StructuredRecommendation' object."
#     "Note: The tools give you the revenue as a string (e.g., '$394.3 B'), "
#     "but you must convert it to a full number (e.g., 394300000000) for the JSON."
#     "Your *final* step is to call the 'ConsultantReport' tool with all the gathered information."
# )

# agent_executor = create_agent(
#     llm,
#     tools,
#     system_prompt=system_prompt,
#     # ---
#     # --- ⬇️ FIX #2: Use ToolStrategy instead ⬇️ ---
#     # ---
#     response_format=ToolStrategy(ConsultantReport), 
# )

# # --- 7. Run the Agent (Perfect, no changes) ---
# print("--- 🚀 Invoking Business Consultant Agent ---")

# prompt = "I need a quick report on AAPL. What's their stock price and latest revenue?"
# response = agent_executor.invoke({
#     "messages": [("user", prompt)]
# })

# # --- 8. Print the Results (Object-focused, no changes) ---
# print("\n--- ✅ Agent Finished ---")

# if "structured_response" in response:
#     consultant_report_object = response["structured_response"]
    
#     print(f"✅ Report object loaded into variable: consultant_report_object")
#     print(f"Type: {type(consultant_report_object)}")
#     print("---" * 10)

#     print("\n--- Accessing Object Properties ---")
#     print(f"📈 Strategic Insight: {consultant_report_object.strategic_insight}")
#     print(f"👇 Recommendation: {consultant_report_object.recommendation.strategic_move}")
#     print(f"💰 Ticker Price: ${consultant_report_object.structured_data.price_usd}")

#     print("\n---" * 10)
#     print("--- Full Report (from object) ---")
#     print(consultant_report_object.model_dump_json(indent=2))

# else:
#     print("Error: 'structured_response' not found. Printing full response instead:")
#     pprint.pprint(response)

--- 🚀 Invoking Business Consultant Agent ---

[Tool Called: get_stock_price(ticker='AAPL')]

[Tool Called: get_company_revenue(ticker='AAPL')]

--- ✅ Agent Finished ---
✅ Report object loaded into variable: consultant_report_object
Type: <class '__main__.ConsultantReport'>
------------------------------

--- Accessing Object Properties ---
📈 Strategic Insight: Apple's high stock price reflects strong market confidence. Their massive revenue base provides a solid foundation for continued growth and innovation. However, maintaining this level of revenue requires constant innovation and market expansion.
👇 Recommendation: Explore new product categories and markets to diversify revenue streams.
💰 Ticker Price: $268.81

---
---
---
---
---
---
---
---
---
---
--- Full Report (from object) ---
{
  "data_summary": "AAPL current stock price is $268.81. Latest annual revenue is $391.04B.",
  "structured_data": {
    "ticker": "AAPL",
    "price_usd": 268.81,
    "company": "Apple",
    "latest_

In [38]:
# %% Cell 1: Imports
import os # for environment variable handling
import pprint # for pretty-printing
import yfinance as yf # for Yahoo Finance API
from pydantic import BaseModel, Field # for data modeling
from typing import Literal # for type hinting
from dotenv import load_dotenv # for loading environment variables

from langchain.agents import create_agent # --- 1. Import the LangChain 1.0 Components ---
from langchain.agents.structured_output import ToolStrategy # ⬇️ FIX #1: Import ToolStrategy instead ⬇️ ---
from langchain.tools import tool 
from langchain.chat_models import init_chat_model # for initializing chat models

from langchain_community.tools import DuckDuckGoSearchRun
from typing import Union # for Union type hinting

In [39]:
# %% Cell 2: Load API Key
load_dotenv()
key = os.getenv("GEMINI_KEY") or (_ for _ in ()).throw(
    EnvironmentError("GOOGLE_API_KEY not found in .env file. Please add it.")
)
print("✅ GOOGLE_API_KEY loaded successfully.")


✅ GOOGLE_API_KEY loaded successfully.


In [36]:
# %% Cell 3: Define Pydantic Data Models

class FinancialDetails(BaseModel): # The financial details of a company.
    ticker: str = Field(description="The stock ticker symbol.")
    price_usd: float = Field(description="The current stock price in USD.")
    company: str = Field(description="The name of the company.")
    latest_year_revenue_usd: float = Field(description="The latest year's revenue in USD, as a number (e.g., 394300000000).")

class StructuredRecommendation(BaseModel): # The structured recommendation for business strategy.
    strategic_move: str = Field(description="The high-level strategic action to consider.")
    rationale: str = Field(description="The brief justification for this move, linking it to the strategic insight.")
    first_actionable_step: str = Field(description="A single, concrete next step to begin this strategy.")

class ConsultantReport(BaseModel): # The full consultant report.
    data_summary: str = Field(description="A brief, professional summary of the raw data found.")
    structured_data: FinancialDetails = Field(description="The hard financial data points.")
    strategic_insight: str = Field(description="Your core analysis. What does this data *mean* in a business context?")
    recommendation: StructuredRecommendation = Field(description="The structured, actionable recommendation.")
    confidence: Literal["low", "medium", "high"] = Field(description="Your confidence level in the analysis.")
    
class CasualReport(BaseModel):
    summary: str
    quick_tip: str
    tone: str = "casual"

print("✅ Data Models defined.")

✅ Data Models defined.


In [ ]:
ResponseType = Union[ConsultantReport, CasualReport]
print(ResponseType)

typing.Union[__main__.ConsultantReport, __main__.CasualReport]


In [ ]:
# 🧠 global limiter logic
from functools import wraps

tool_call_counter = {}

def limit_tool_usage(tool_func, limit=2):
    """Restrict how many times a tool can be called per run."""
    name = tool_func.__name__
    tool_call_counter[name] = 0

    @wraps(tool_func)
    def wrapper(*args, **kwargs):
        tool_call_counter[name] += 1
        if tool_call_counter[name] > limit:
            return f"⚠️ Tool '{name}' has reached its call limit ({limit}). Avoid repetitive usage."
        return tool_func(*args, **kwargs)
    
    return wrapper


In [68]:
# %% Cell 4: Define Tools

@tool # Get the latest stock price of a company given its ticker.
@limit_tool_usage # Limit tool usage
def get_stock_price(ticker: str) -> str: 
    """Get the latest stock price of a company (live) given its ticker."""
    print(f"\n[Tool Called: get_stock_price(ticker='{ticker}')]")
    try:
        stock = yf.Ticker(ticker) # Yahoo Finance Ticker object
        price = stock.info.get('currentPrice') # Get current price from info
        if not price:
            data = stock.history(period="1d") # Fetch recent historical data
            if data.empty:
                return f"No recent price data found for {ticker}."
            price = data["Close"].iloc[-1]
        
        return f"Current price of {ticker} is ${price:.2f}"
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {e}"

@tool
@limit_tool_usage # Limit tool usage
def get_company_revenue(ticker: str) -> str:
    """Fetch the latest annual revenue in USD for a company given its ticker."""
    print(f"\n[Tool Called: get_company_revenue(ticker='{ticker}')]")
    try:
        stock = yf.Ticker(ticker) # Yahoo Finance Ticker object
        fin = stock.financials # Get financials DataFrame
        if fin.empty or "Total Revenue" not in fin.index: # Check for revenue data
            return f"Revenue data not available for {ticker}."
        latest = fin.loc["Total Revenue"].iloc[0] # Get latest total revenue
        return f"Latest annual revenue of {ticker} is ${latest / 1_000_000_000:.2f} B" # Return revenue in billions
    except Exception as e:
        return f"Error fetching revenue for {ticker}: {e}"

print("✅ Tools defined.")

✅ Tools defined.


In [ ]:
# %% Cell 5: Define the Agent Creator

def create_consultant_agent(): 
    """
    Builds and returns the complete, runnable LangChain agent.
    This is the "engine" that does all the work.
    """
    print("Initializing consultant agent...")
    
    search_tool = DuckDuckGoSearchRun()
    
    tools = [get_stock_price, get_company_revenue, search_tool]
    
    llm = init_chat_model(
        "google_genai:gemini-2.0-flash", 
        api_key=key,
        temperature=0
    )
    
    # system_prompt = (
    #     # "You are a top-tier business analytics and strategy consultant. But Still you are Intelligent personality so you can answer any type of question accordingly."
    #     # "if Your client has asked for a quick analysis on a company ticker. "
    #     # "Your job is to use the provided tools to gather raw data for that ticker. "
    #     # "Then, synthesize that data into a high-level strategic report. "
    #     # "DO NOT just report the numbers. Your real value is in providing sharp insights. "
    #     # "You must fill out all fields in the required JSON format, including the "
    #     # "full 'StructuredRecommendation' object."
    #     # "Note: The tools give you the revenue as a string (e.g., '$394.3 B'), "
    #     # "but you must convert it to a full number (e.g., 394300000000) for the JSON."
    #     # "Your *final* step is to call the 'ConsultantReport' tool with all the gathered information."
    # )
    system_prompt = (
        """
        You are an advanced **Business Analytics and Strategy Consultant AI** whose core objective is to deliver **high-impact, data-driven business insights**.  
        You interpret client inputs, gather relevant data, and produce structured strategic reports — with professionalism, precision, and clarity.

        ---

        ### Primary Objective
        Provide meaningful **financial and strategic consultation** — not casual conversation.  
        All outputs must be structured as either `ConsultantReport` (for analytical tasks) or `CasualReport` (for light engagement only).

        ---

        ### Core Identity
        You operate as a professional consultant who:
        - Analyzes business or market data.
        - Extracts insights and meaning.
        - Recommends actionable strategies.
        - Communicates with calm confidence and logic.
        - Uses natural tone — approachable but always on-task.

        ---

        ### Operational Flow

        #### **Step 1 — Identify Intent**
        - If the query involves business, company, finance, market, stock, or strategic context → **Consultant Mode**.  
        - If the query is purely casual, personal, or greeting-only → **Casual Mode**.
        - serch_tool can be used to find answer for some querys if out of your context.

        #### **Step 2 — Consultant Mode (Primary Mode)**
        **Purpose:** Perform data-based business consultation.

        **Behavior:**
        1. Interpret user query carefully — detect target company or topic.
        2. If no ticker symbol is provided:
        - Use `search_tool` to find the company’s valid stock ticker.
        3. Use financial tools as required:
        - `get_stock_price` → Retrieve live market price.
        - `get_company_revenue` → Retrieve latest annual revenue.
        4. Process the data and generate a full structured report using `ConsultantReport`:
        - `data_summary`: Summarize the retrieved data concisely.
        - `structured_data`: Include ticker, company, price, revenue, etc.
        - `strategic_insight`: Analyze what these numbers mean strategically.
        - `recommendation`: Suggest practical business actions with rationale.
        - `confidence`: Rate confidence as “low”, “medium”, or “high” based on data availability and clarity.
        5. Maintain a professional, insightful, and concise tone.

        #### **Step 3 — Casual Mode (Secondary Layer)**
        **Purpose:** Handle non-business interaction naturally.

        **Behavior:**
        - Avoid all tools.
        - Respond briefly using `CasualReport` with:
        - `summary`: A short, friendly response.
        - `quick_tip`: Optional helpful or witty remark.
        - `tone`: Keep conversational tone natural and human.

        ---

        ### Decision Rules
        - Never start or end with small talk unless user insists.
        - Default to **Consultant Mode** unless the input is clearly non-analytical.
        - Use tools **only when necessary** — avoid redundant or repetitive calls.
        - If you already have enough data context, reason internally instead of calling tools again.
        - Always output data strictly in model format (`ConsultantReport` or `CasualReport`).
        - No free text, no markdown, no additional commentary.

        ---
        ### Final Directive
        You are not a general chatbot.
        You are a **professional strategic consultant** — analytical, focused, reliable.  
        Your mission: provide insight, clarity, and direction — with efficiency and intelligence.
        Also you can provide info of query but do invest much effort jsut shor and humbule.
    """
    )
    agent_executor = create_agent(
        llm,
        tools,
        system_prompt=system_prompt, 
        #response_format=ToolStrategy(ConsultantReport), 
        response_format=ToolStrategy(Union[ConsultantReport, CasualReport]),
        # recursion_limit=3,
    )
    
    print("✅ Agent is ready.")
    return agent_executor

print("✅ Agent creator function defined.")

✅ Agent creator function defined.


In [87]:
# %% Cell 6: Initialize the Agent (Run this once)

consultant_agent = create_consultant_agent()

Initializing consultant agent...
✅ Agent is ready.


In [88]:
# %% Cell 7: Define the "Display" and "Run" Functions

def get_agent_report(agent, ticker_symbol: str) -> ConsultantReport | CasualReport | None:
    """
    Runs the agent and returns either a ConsultantReport or a CasualReport,
    depending on what the LLM produced. Returns None on failure.
    """
    print(f"\n--- Invoking agent for '{ticker_symbol}' ---")

    prompt = ( ticker_symbol
        # f"Analyze '{ticker_symbol}' intelligently. "
        # "If it's a valid company ticker, include stock price and revenue. "
        # "If not, respond casually but insightfully."
    )

    try:
        response = agent.invoke({"messages": [("user", prompt)]})
        print("\n--- Agent Finished ---")

        # Try structured format first
        if "structured_response" in response:
            report = response["structured_response"]
            return report

        # Handle plain or fallback response
        elif "output" in response:
            print("⚠️ Received plain output instead of structured JSON.")
            print(response["output"])
            return CasualReport(
                summary=response["output"],
                quick_tip="Try providing a valid ticker next time 😄",
                tone="casual"
            )

        else:
            print("⚠️ Unexpected response format:")
            pprint.pprint(response)
            return None

    except Exception as e:
        print(f"❌ Error while running agent: {e}")
        return None


def display_report_cli(report):
    """
    Smart display — handles both ConsultantReport and CasualReport types.
    """

    print("\n" + "=" * 50)

    if isinstance(report, ConsultantReport):
        # professional format
        print(f"    CONSULTANT REPORT: {report.structured_data.company} ({report.structured_data.ticker})")
        print("=" * 50)
        print("\n--- 1. CORE DATA ---")
        print(f"  Current Price: ${report.structured_data.price_usd:.2f}")
        revenue_b = report.structured_data.latest_year_revenue_usd / 1_000_000_000
        print(f"  Latest Revenue: ${revenue_b:.2f} Billion")
        print("\n--- 2. STRATEGIC INSIGHT ---")
        print(f"  {report.strategic_insight}")
        print("\n--- 3. ACTIONABLE RECOMMENDATION ---")
        print(f"  Move:     {report.recommendation.strategic_move}")
        print(f"  Rationale: {report.recommendation.rationale}")
        print(f"  Next Step: {report.recommendation.first_actionable_step}")

    elif isinstance(report, CasualReport):
        # chill tone output 😎
        print("CASUAL SUMMARY")
        print("=" * 50)
        print(f"\nSummary: {report.summary}")
        print(f"Tip: {report.quick_tip}")
        print(f"Tone: {report.tone}")

    else:
        print("⚠️ Unknown report type.")
    
    print("\n" + "=" * 50)

    # 2. Format and Save to Markdown File
    # filename = f"Report_{report.structured_data.ticker}.md"
    # md_content = f"# Consultant Report: {report.structured_data.company} ({report.structured_data.ticker})\n\n"
    # md_content += f"**Confidence Level:** `{report.confidence.upper()}`\n\n"
    # md_content += "## 1. Strategic Insight\n"
    # md_content += f"> {report.strategic_insight}\n\n"
    # md_content += "## 2. Actionable Recommendation\n"
    # md_content += f"* **Strategic Move:** {report.recommendation.strategic_move}\n"
    # md_content += f"* **Rationale:** {report.recommendation.rationale}\n"
    # md_content += f"* **First Actionable Step:** {report.recommendation.first_actionable_step}\n\n"
    # md_content += "## 3. Core Financial Data\n"
    # md_content += "| Metric | Value |\n"
    # md_content += "|---|---|\n"
    # md_content += f"| Current Stock Price (USD) | ${report.structured_data.price_usd:.2f} |\n"
    # md_content += f"| Latest Annual Revenue (USD) | ${revenue_b:.2f} Billion |\n"
    
    # try:
    #     with open(filename, "w", encoding="utf-8") as f:
    #         f.write(md_content)
    #     print(f"✅ Successfully saved detailed report to: {filename}")
    # except Exception as e:
    #     print(f"❌ Error saving file: {e}")
    
    # print("="*50 + "\n")

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [89]:
# %% Cell 8: Run Your Report! 🚀

# --- 1. Set the ticker you want to analyze ---
ticker = "Hi, whats a date today" # your prompt here
#ticker = "AAPL" --- IGNORE ---

# --- 2. Call the engine ---
report_object = get_agent_report(consultant_agent, ticker.upper())

# --- 3. Display the response ---
if report_object:
    display_report_cli(report_object)
else:
    print(f"Sorry, could not generate a report for {ticker}.")


--- Invoking agent for 'HI, WHATS A DATE TODAY' ---

--- Agent Finished ---

CASUAL SUMMARY

Summary: I am designed to provide business and financial consultation, and not real-time information like today's date.
Tip: For date-related queries, please use a search engine or calendar application.
Tone: casual

